## Determination of Binary Interaction Parameters for Ibuprofen and CO2 from Solubility Data 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Python_Projects/solubility_in_sc_CO2

/content/gdrive/MyDrive/Python_Projects/solubility_in_sc_CO2


In [ ]:
import numpy as np 
import sympy as sp
from sympy import pi
from sympy.abc import i
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from IPython.display import display
from sympy.interactive import init_printing
init_printing(use_latex=True)
from sympy import pprint

In [ ]:
! pip install pyrenn

In [ ]:
from fugacity_coeff import fugacity_scf
from mixing_parameters import a_mix, a_PRK, alpha, b_mix, b_PRK, a_star, b_star, a_m_cosolvent, a_s_cosolvent, b_m_cosolvent, b_s_cosolvent
from Z_prk import Z_PRK
from P_sat import P_sat
from find_solubility import find_solubility_cosol, find_solubility
from results import write_to_excel
from evaluate_model import AARD

### Solubility Data of Ibuprofen-CO2 at 313.15 K 

Solubility data is obtained from <a href=http://www.ijche.com/article_10327_dc073ba6acbffc77feb86f32eb5283f6.pdf>this study.<a>

In [ ]:
y_expe = np.array([0.000198, 0.000573, 0.000766, 0.001393, 0.001596, 0.002142, 0.002287, 0.003019, 0.003261])
P_expe = np.array([9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13])

### Inputs

In [ ]:
T_num = 313.15 # in Kelvins
R_num = 8.314 #Pa . m3 . mol-1 . K-1

In [ ]:
T_c_ibu = 749.52
P_c_ibu = 2315000
w_ibu = 0.820
v_solid_num = 182.14/10**(6) # m3/mol

In [ ]:
P_sub_num = 0.0897

In [ ]:
T_c_co2 = 304.25
P_c_co2 = 7377300 # Pa
w_co2 = 0.225

In [ ]:
T_c_eth = 516.2
P_c_eth= 6140000 # Pa
w_eth= 0.635

### Calculations

In [ ]:
alpha_co2 = alpha(w_co2, T_num, T_c_co2)
alpha_ibu = alpha(w_ibu, T_num, T_c_ibu)

a_PRK_CO2 = a_PRK(P_c_co2, T_c_co2, R_num) * alpha_co2
a_PRK_ibu = a_PRK(P_c_ibu, T_c_ibu, R_num) * alpha_ibu

b_PRK_CO2 = b_PRK(P_c_co2, T_c_co2, R_num)
b_PRK_ibu = b_PRK(P_c_ibu, T_c_ibu, R_num)

### Minimization of Average Absolute Relative Deviation AARD(%)

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
def objective_AARD(x):
  k_12 = x[0]
  k_num = np.array([[0,k_12], [k_12, 0]])

  l_12 = x[1]
  l_num = np.array([[0,l_12], [l_12, 0]])

  print("Calculation of solubility using PRK_EOS vdw1")
  y_calc_num = [find_solubility(v_solid_num, p*10**6, P_sub_num, R_num, T_num, a_PRK_CO2, a_PRK_ibu, b_PRK_CO2, b_PRK_ibu, k_num, l_num) for p in P_expe]
  y_calculated = np.array(y_calc_num)
  print("\nDone!\n")
  
  
  N, i = sp.symbols('N i')
  y_cal = sp.IndexedBase('y_cal')
  y_exp = sp.IndexedBase('y_exp')

  print("Calculation of AARD(%)")
  AARD_1 = 100/N*sp.Sum(sp.Abs(y_cal[i] - y_exp[i])/y_exp[i], (i,0,N-1)).doit()
  AARD_fun = sp.lambdify([y_cal, y_exp, N], AARD_1, 'numpy')
  AARD_num = AARD_fun(y_calculated, y_expe, len(y_expe))
  print("\nDone!\n")
  
  return AARD_num

In [ ]:
k_12_0 = 0
k_num = np.array([[0,k_12_0], [k_12_0, 0]])

In [ ]:
l_12_0 = 0
l_num = np.array([[0,l_12_0],[l_12_0,0]])

In [ ]:
x0 = np.array([k_12_0, l_12_0], dtype=object)

In [ ]:
sol = minimize(objective_AARD, x0, method='Nelder-Mead')

In [ ]:
sol

 final_simplex: (array([[0.14356282, 0.16855572],
       [0.14352755, 0.16846524],
       [0.14353309, 0.16847924]]), array([6.74143739, 6.74144858, 6.74145967]))
           fun: 6.741437386819106
       message: 'Optimization terminated successfully.'
          nfev: 214
           nit: 112
        status: 0
       success: True
             x: array([0.14356282, 0.16855572])

Binary interaction parameters, $k_{12}$ and $l_{12}$, were found as 0.1436, and 0.1685, respectively.